#### Agentic Systems are of 2 types

* **Workflows** : are systems where LLM's and tools are orchastrated through predefined code paths
* **Agents** : are systems where LLM's dynamically direct their own processes and tool usage,maintaining control over how they accomplish tasks.

#### 5 Workflow design patterns:

    1) **Prompt Chaining** : Decompose into fixed sub-tasks
        IN-->LLM1-->Gate(code)-->LLM2-->LLM3-->OUT
    2) **Routing** : Direct an input into a specialized sub task, ensuring separation of concerns
        IN-->LLM router-->parallel(LLM1+LLM2+LLM3...)-->OUT
    3) **Parallelization** : Breaking down tasks and running multiple subtasks concurrently, same like above but here a logic send to parallel   flows,unlike an LLM earlier
        IN-->Coordinator(code)-->parallel(LLM1+LLM2+LLM3...)-->Aggregator(code)-->OUT
    4) **Orchestrator-worker** : Complex tasks are broken down dynamically and combined
        IN-->Orchestrator(LLM)-->parallel(LLM1+LLM2+LLM3...)-->Synthesizer(LLM)-->OUT
    5) **Evaluator-optimizer** : LLM output is validated by another LLM
        IN-->LLM generation<==[solution/rejection]==>LLM evaluator==[accepted]==>OUT


#### By contrast, **Agents**
    1) Open-ended
    2) Feedback loops
    3) No fixed paths

    Human-->LLM call<==[Action/Feedback]==>Environment
                ||
               Stop


In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

if openai_api_key:
    print(f"OpenAI API key is set: {openai_api_key[:2]}....")
else:
    print("OpenAI API key is not set")

if google_api_key:
    print(f"Google API key is set: {google_api_key[:2]}....")
else:
    print("Google API key is not set")

if groq_api_key:
    print(f"Groq API key is set: {groq_api_key[:2]}....")
else:
    print("Groq API key is not set")

OpenAI API key is set: sk....
Google API key is set: AI....
Groq API key is set: gs....


In [4]:
#test all the keys
messages = [{"role": "user", "content": "What is 2 + 2?"}]
groq_client = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
gemini_client = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
openai_client = OpenAI()

groq_response = groq_client.chat.completions.create(
    model="llama3-8b-8192",
    messages=messages,
)
gemini_response = gemini_client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=messages,
)

openai_response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

print(f"Groq : {groq_response.choices[0].message.content}")
print(f"Google : {gemini_response.choices[0].message.content}")
print(f"OpenAI : {openai_response.choices[0].message.content}")






Groq : The answer to 2 + 2 is 4.
Google : 2 + 2 = 4

OpenAI : 2 + 2 equals 4.


In [5]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."

messages = [{"role": "user", "content": request}]
print(messages[0]["content"])









Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.


In [6]:
#lets form the question

openai_question = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages = messages
)


question = openai_question.choices[0].message.content
print(question)


If you could redesign human cognition using a combination of traits from various animal species, what traits would you select, which species would you draw from, and how would these changes impact aspects of human society like communication, creativity, and problem-solving?


In [7]:
models = []
answers = []
messages = [{"role": "user", "content": question}]

In [8]:
client_model = {"gemini-2.0-flash": gemini_client,"gpt-4o-mini": openai_client, "llama3-8b-8192": groq_client}

In [9]:
for model, client in client_model.items():
    model_response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    model_answer = model_response.choices[0].message.content

    models.append(model)
    answers.append(model_answer)
    print("================================================")
    display(Markdown(f"**{model}**\n{model_answer}"))
    print("================================================")


**gemini-2.0-flash**
This is a fascinating thought experiment! Here's my attempt at redesigning human cognition, drawing inspiration from the animal kingdom and considering its potential societal impact:

**Goal:** To create a more empathetic, collaborative, and resourceful human cognitive architecture while mitigating some of our current flaws (biases, impulsivity, short-sightedness).

**Redesigned Cognitive Traits:**

1.  **Enhanced Olfactory Perception (from Dogs/Elephants):**

    *   **Species:** Dogs (for discrimination), Elephants (for long-distance sensing)
    *   **Why:** While humans have lost a significant portion of olfactory capability, smell is deeply connected to memory and emotion. Amplified and more nuanced olfactory abilities could:
        *   **Impact on Communication:** Augment communication through subtle scent cues (fear, stress, health status), leading to greater emotional intelligence. Imagine sensing anxiety in a colleague during a negotiation.
        *   **Impact on Creativity:** Unlock new sensory experiences and associations, influencing artistic expression and culinary innovation. Think of "smell composers" creating olfactory symphonies.
        *   **Impact on Problem-solving:** Improve threat detection (e.g., recognizing the scent of dangerous chemicals or diseases), enable better environmental monitoring, and even aid in tracking and locating resources.
        *   **Ethical Concerns:** Privacy concerns become heightened.  Would we need "olfactory encryption" to mask our emotional state?

2.  **Synesthesia Integration (from Synesthetic Animals, unknown):**

    *   **Species:** Many animals are thought to experience synesthesia (e.g., certain insects).  It's difficult to pinpoint a specific species due to the subjective nature of the experience.
    *   **Why:**  Synesthesia, the blending of senses (e.g., seeing sounds, tasting colors), can create novel connections and enhance creative thinking.
        *   **Impact on Communication:** Language becomes more multi-sensory, with words evoking images, sounds, and even tastes. Art becomes more accessible to everyone as people naturally experience the senses together.
        *   **Impact on Creativity:**  Explosive creativity as ideas form across sensory boundaries.  New art forms would emerge that leverage the cross-sensory experience.  New solutions to problems could be unlocked through multisensory thinking.
        *   **Impact on Problem-solving:**  Enhanced pattern recognition and abstract thinking. Someone might "see" a mathematical equation as a color pattern that reveals its solution.
        *   **Challenges:**  Potential for sensory overload and difficulty in focusing.  Would require development of cognitive filters to manage the sensory input.

3.  **Collective Intelligence and Communication (from Ants/Bees/Wolves):**

    *   **Species:** Ants, Bees (for decentralized decision-making), Wolves (for cooperative hunting)
    *   **Why:**  Humans often struggle with large-scale collaboration.  Integrating swarm intelligence and clear communication protocols could:
        *   **Impact on Communication:** Develop highly efficient and transparent communication systems. A global digital network could facilitate instant dissemination of information and collaborative problem-solving.
        *   **Impact on Creativity:**  Enable collective brainstorming and innovation on a massive scale. Complex projects could be tackled by interconnected teams with specialized knowledge, working towards a common goal.
        *   **Impact on Problem-solving:** Improve crisis response (natural disasters, pandemics) through coordinated efforts.  Resource allocation and information sharing would be significantly more efficient.
        *   **Risks:**  Potential for hive-mind mentality and suppression of individual thought.  Need to maintain individuality and critical thinking within the collective. The ability to "disconnect" would be crucial.

4.  **Episodic Future Thinking (from Scrub Jays):**

    *   **Species:** Scrub Jays (and other corvids)
    *   **Why:** Humans often act impulsively, neglecting long-term consequences. Scrub Jays "cache" food based on anticipated future needs.  Enhanced episodic future thinking would:
        *   **Impact on Communication:**  Long-term planning and vision become central to communication. Persuasion focuses on future benefits and collective well-being rather than immediate gratification.
        *   **Impact on Creativity:**  Future-oriented design and innovation. Artists and engineers would focus on creating sustainable solutions that benefit generations to come.
        *   **Impact on Problem-solving:** Improve resource management, environmental stewardship, and long-term strategic planning.  Less likely to make decisions that jeopardize future well-being.
        *   **Downside:**  Could lead to excessive anxiety about the future.  Balance is key.

5.  **Spatial Awareness and Navigation (from Birds/Dolphins):**

    *   **Species:** Migratory Birds (for long-distance navigation), Dolphins (for echolocation and 3D spatial understanding)
    *   **Why:** Enhanced spatial abilities could drastically improve problem-solving, memory, and creativity.
        *   **Impact on Communication:** Could lead to the development of new languages and communication systems that are spatially oriented, allowing for more precise and nuanced descriptions of complex environments and relationships.
        *   **Impact on Creativity:** Spatial reasoning is crucial to understanding complex systems and processes. Scientists and engineers could design more effective and innovative solutions to complex problems by being able to visualize and manipulate spatial information more effectively.
        *   **Impact on Problem-solving:** Improved navigation and optimization of resources. Better understanding of complex systems (e.g., climate models, urban planning).
        *   **Considerations:** Could lead to a greater emphasis on visual and spatial thinking, potentially marginalizing other forms of intelligence.

**Societal Implications (General):**

*   **Greater Empathy & Cooperation:** A more collaborative and emotionally attuned society, reducing conflict and promoting social harmony.
*   **Sustainable Future:**  Increased focus on long-term planning and environmental stewardship, leading to a more sustainable and equitable future.
*   **Creative Renaissance:**  An explosion of creativity and innovation across all fields, driven by enhanced sensory perception, cross-sensory thinking, and collective intelligence.
*   **Ethical Challenges:**  New ethical dilemmas arise concerning privacy, individuality, and the potential for manipulation.
*   **Cognitive Diversity:** It is important to maintain a diversity of cognitive styles and abilities.  Too much homogeneity could stifle innovation.

**Conclusion:**

Redesigning human cognition is a complex and ethically fraught endeavor. While incorporating traits from other species could potentially lead to significant improvements in our ability to communicate, create, and solve problems, it is crucial to carefully consider the potential downsides and ensure that individual autonomy and cognitive diversity are preserved. This thought experiment highlights the incredible potential of the animal kingdom to inspire new approaches to human problem-solving and the importance of ethical considerations when contemplating such profound changes.


**gpt-4o-mini**
Redesigning human cognition by drawing from various animal species offers an intriguing exploration into the capabilities of different traits. Here are some traits that could be selected and the potential impacts on society:

1. **Memory and Spatial Navigation (Dolphins)**
   - **Trait:** Dolphins are known for their complex social structures and advanced problem-solving abilities, aided by their excellent memory and spatial awareness.
   - **Impact:** Enhanced memory and spatial navigation could lead to improved collaboration and teamwork. People could easily recall facts, relationships, and spatial layouts, enhancing training methods, education systems, and urban planning.

2. **Sensory Perception and Communication (Dogs)**
   - **Trait:** Dogs possess acute senses, particularly smell and hearing, and they are adept at reading human emotions and intentions through vocal tones and body language.
   - **Impact:** With heightened emotional intelligence and sensory awareness, human communication would become more nuanced and empathetic. Conflict resolution would improve as individuals would better understand each other’s needs and feelings, fostering more compassionate societies.

3. **Innovative Problem-Solving (Crows and Parrots)**
   - **Trait:** Crows and parrots demonstrate remarkable problem-solving skills and tool use, indicating high levels of intelligence and adaptability.
   - **Impact:** Incorporating such cognitive abilities could enhance creativity and innovation. Society might see a surge in technological advancements, artistic expressions, and new ways of tackling issues such as climate change and social inequality due to the more innovative approaches from individuals.

4. **Collaborative Intelligence (Ants)**
   - **Trait:** Ants exhibit highly organized and cooperative behavior, emphasizing collective problem-solving and resource allocation.
   - **Impact:** Instilling a greater sense of community and collective action could transform human societies into more cooperative entities. This could lead to efficient resource distribution, enhanced community resilience, and solutions to global challenges through shared knowledge and unified efforts.

5. **Adaptability and Learning (Octopus)**
   - **Trait:** Octopuses are known for their adaptability, learning through observation, and ability to navigate complex environments.
   - **Impact:** Enhanced adaptability could make individuals more tolerant of change, facilitating smoother transitions in societies faced with rapid technological shifts or cultural changes. Lifelong learning could become the norm, reducing resistance to new ideas and fostering a culture of continuous improvement.

6. **Intuition and Instinct (Cats)**
   - **Trait:** Cats possess strong instincts and a keen intuition about their environment.
   - **Impact:** Improved intuition could lead individuals to make better decisions, relying less on purely logical analysis and more on holistic perceptions. This might foster creativity in strategic sectors such as business strategy and art.

7. **Group Cohesion and Navigation (Wolves)**
   - **Trait:** Wolves demonstrate strong social bonds, and coordinated group dynamics, which enhance their hunting success and survival.
   - **Impact:** Human relationships would likely be more collaborative and group-oriented, leading to improved social structures, greater collective welfare, and possibly a decrease in loneliness and isolation issues prevalent in modern society.

In combination, these traits could result in a society characterized by heightened empathy, innovative problem-solving, robust collaboration, adaptability to change, and intuitive decision-making. The arts might flourish, technological advancements could accelerate, and global cooperation could become more seamless as humans become more attuned to each other and their environments. Balancing individual desires with community well-being would be pivotal in shaping a prosperous and harmonious society.

**llama3-8b-8192**
What a fascinating question! Redesigning human cognition by combining traits from various animal species would indeed be a complex and intriguing endeavor. Here's a hypothetical scenario where I select and combine traits from different species to create a unique cognitive profile, examining its potential impact on human society.

**Traits selection:**

1. **Omnidirectional attention** from octopuses: Octopuses can focus on multiple stimuli simultaneously, making them excellent at multitasking. Humans could benefit from this trait, enabling them to process and respond to various information sources concurrently.
2. **Spatial reasoning** from bees: Bees excel at navigating complex spatial environments, remembering the location of food sources, and communicating this information to their colony. Humans could incorporate this trait to improve their ability to understand complex spatial relationships and problem-solve in geometrically demanding situations.
3. **Pattern recognition** from dolphins: Dolphins possess exceptional pattern recognition abilities, allowing them to identify and track prey, recognize friendly and hostile individuals, and even understand human language. Humans could integrate this trait to improve their ability to recognize patterns, decipher hidden information, and make connections between seemingly unrelated concepts.
4. **Adaptive learning** from ants: Ants are renowned for their ability to adapt to changing environments and learn from experience. Humans could incorporate this trait to enhance their capacity for rapid learning, problem-solving, and adapting to new situations.
5. **Efficient communication** from birds: Birds have evolved sophisticated communication systems, including visual, auditory, and kinetic cues. Humans could benefit from adopting more efficient and nuanced communication methods, reducing misunderstandings and improving collaboration.

**Cognitive profile:**

With these traits combined, the redesigned human cognitive profile would possess the following characteristics:

* **Multitasking**: The ability to focus on multiple stimuli simultaneously, allowing individuals to process and respond to various information sources concurrently.
* **Spatial intelligence**: Enhanced spatial reasoning, enabling individuals to understand complex geometric relationships and navigate challenging environments.
* **Pattern recognition**: Improved ability to identify and decipher patterns, facilitating connections between seemingly unrelated concepts and enhanced problem-solving capacity.
* **Adaptive learning**: Rapid adaptation to changing environments and learning from experience, facilitating efficient problem-solving and response to new situations.
* **Efficient communication**: Nuanced and efficient communication methods, reducing misunderstandings and improving collaboration.

**Impact on human society:**

This redesigned cognitive profile would likely have significant impacts on various aspects of human society:

* **Communication**: Improved communication methods would enhance collaboration, reduce misunderstandings, and facilitate more effective problem-solving. This could lead to increased efficiency in industries, improved decision-making in governments, and more harmonious personal relationships.
* **Creativity**: Enhanced pattern recognition and spatial intelligence could lead to breakthroughs in art, science, and technology. The ability to connect seemingly unrelated concepts could result in innovative solutions to complex problems.
* **Problem-solving**: The adaptive learning and pattern recognition abilities could lead to more efficient and effective problem-solving, enabling humans to respond to complex challenges more quickly and creatively.
* **Education**: The multitasking and adaptive learning traits could revolutionize traditional educational approaches, enabling students to learn more rapidly and effectively, and making lifelong learning a more accessible and enjoyable experience.
* **Workforce**: The efficient communication and spatial intelligence traits could increase productivity and collaboration in industries such as construction, architecture, and logistics, while the adaptive learning and pattern recognition abilities could enable workers to respond more effectively to changing circumstances and complex problems.

**Challenges and limitations:**

While this redesigned cognitive profile would bring many benefits, there are also potential challenges and limitations to consider:

* **Cultural and social adjustments**: This new cognitive profile would require significant cultural and social adaptations, as traditional communication and problem-solving methods would need to be revised.
* **Neurological and psychological impacts**: The introduction of these new traits could lead to concerns about neurological and psychological well-being, particularly if individuals struggle to accommodate the changes.
* **Ethical considerations**: The potential for exploiting or using these traits for malicious purposes would need to be carefully considered and addressed.

In conclusion, redesigning human cognition using a combination of traits from various animal species is a fascinating thought experiment that highlights the potential for optimizing human cognitive abilities. While there are challenges and limitations to consider, the benefits of this redesigned cognitive profile would likely have significant impacts on communication, creativity, and problem-solving, ultimately enhancing human society as a whole.

In [10]:
for model,answer in zip(models,answers):
    print(f"Competitor: {model} \n\n\n{answer[:500]}")
    print("================================================")

Competitor: gemini-2.0-flash 


This is a fascinating thought experiment! Here's my attempt at redesigning human cognition, drawing inspiration from the animal kingdom and considering its potential societal impact:

**Goal:** To create a more empathetic, collaborative, and resourceful human cognitive architecture while mitigating some of our current flaws (biases, impulsivity, short-sightedness).

**Redesigned Cognitive Traits:**

1.  **Enhanced Olfactory Perception (from Dogs/Elephants):**

    *   **Species:** Dogs (for disc
Competitor: gpt-4o-mini 


Redesigning human cognition by drawing from various animal species offers an intriguing exploration into the capabilities of different traits. Here are some traits that could be selected and the potential impacts on society:

1. **Memory and Spatial Navigation (Dolphins)**
   - **Trait:** Dolphins are known for their complex social structures and advanced problem-solving abilities, aided by their excellent memory and spatial awareness.


In [19]:
all_answers = ""


for index,answer in enumerate(answers):
    all_answers += f"# Response from competitor {index+1}\n\n"
    all_answers += answer + "\n\n"


In [20]:
print(all_answers)

# Response from competitor 1

This is a fascinating thought experiment! Here's my attempt at redesigning human cognition, drawing inspiration from the animal kingdom and considering its potential societal impact:

**Goal:** To create a more empathetic, collaborative, and resourceful human cognitive architecture while mitigating some of our current flaws (biases, impulsivity, short-sightedness).

**Redesigned Cognitive Traits:**

1.  **Enhanced Olfactory Perception (from Dogs/Elephants):**

    *   **Species:** Dogs (for discrimination), Elephants (for long-distance sensing)
    *   **Why:** While humans have lost a significant portion of olfactory capability, smell is deeply connected to memory and emotion. Amplified and more nuanced olfactory abilities could:
        *   **Impact on Communication:** Augment communication through subtle scent cues (fear, stress, health status), leading to greater emotional intelligence. Imagine sensing anxiety in a colleague during a negotiation.
     

In [21]:
judge = f"""You are judging a competition between {len(models)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{all_answers}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [22]:
judge_messages = [{"role": "user", "content": judge}]

In [25]:
judge_answer = openai_client.chat.completions.create(
    model="o3-mini",
    messages=judge_messages

)

result = judge_answer.choices[0].message.content
print(result)






{"results": ["1", "3", "2"]}


In [26]:
results_dict = json.loads(result)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = models[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

Rank 1: gemini-2.0-flash
Rank 2: llama3-8b-8192
Rank 3: gpt-4o-mini
